In [2]:
import boto3
import pandas as pd; pd.set_option('Display.max_columns',100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

s3 = boto3.resource('s3')
bucket_name = 'christran-data445-bucket'
bucket = s3.Bucket(bucket_name)

file_key ='framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#read the data file 
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
##removing NA observations
heart = heart.dropna()

In [4]:
##Defining the input and target varaiable
X = heart[['age', 'currentSmoker','totChol','BMI', 'heartRate']]
Y = heart['TenYearCHD']

#splitting
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

In [5]:
logit_md = LogisticRegression().fit(X_train, Y_train)

#predict
logit_pred = logit_md.predict_proba(X_test)[:,1]
logit_pred

array([0.12905588, 0.10090606, 0.12987432, 0.20536019, 0.10227776,
       0.24934518, 0.14262124, 0.08955721, 0.33383219, 0.062779  ,
       0.11040729, 0.08249861, 0.21982872, 0.20048631, 0.23848594,
       0.15259156, 0.30139851, 0.07995709, 0.03682269, 0.06144527,
       0.13203999, 0.2464403 , 0.13079394, 0.3756871 , 0.08689802,
       0.05870849, 0.17694927, 0.2194222 , 0.1437173 , 0.10932593,
       0.15204054, 0.0589373 , 0.17489378, 0.06369352, 0.35431645,
       0.05546568, 0.09316047, 0.10216488, 0.05170143, 0.1276683 ,
       0.24359109, 0.07902467, 0.1906502 , 0.14558366, 0.22319246,
       0.07502809, 0.04219325, 0.16984469, 0.37361003, 0.36739546,
       0.26510588, 0.06911374, 0.08737271, 0.11672327, 0.2794374 ,
       0.22209607, 0.20640825, 0.33461974, 0.20264995, 0.07224652,
       0.35561554, 0.1460948 , 0.11990203, 0.17459739, 0.09668983,
       0.04860407, 0.08690489, 0.16471709, 0.17482432, 0.11686352,
       0.11477637, 0.17000361, 0.16642497, 0.08392425, 0.08205

In [7]:
#changing likelihood table 
logit_label = np.where(logit_pred < 0.25 , 0, 1)
logit_label

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [8]:
#confusion matrix
confusion_matrix(Y_test, logit_label)

array([[545,  84],
       [ 71,  32]])

In [9]:
#computing accuracy
accuracy_score(Y_test, logit_label)

0.7882513661202186